In [36]:
import os as os
import requests as req
import shutil as shutil
import zipfile as zipfile
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [37]:
def data_directories():
    """
    Funcao de criacao das pastas para armazenamento dos dados, caso nao existam.
    
    """

    # Diretorios para armazenamento dos dados coletados
    directories = ['..\\data\\01-collected'
                  ,'..\\data\\02-cleaned'
                  ,'..\\data\\03-organized'
                  ,'..\\data\\04-standardized']
    
    # Log: Mensagem de inicio da criacao dos diretorios
    print(str(datetime.now()) + ': Criacao dos diretorios para armazenamento dos dados iniciada.')
    
    # Para cada diretorio na lista de diretorios
    for directory in directories:
        
        # Verifica se o diretorio existe
        if not os.path.exists(directory):
            
            # Cria o diretorio
            os.makedirs(directory)
            
            # Log: Mensagem de criacao do diretorio
            print(str(datetime.now()) + ': Diretorio ' + directory + ' criado.')
        
        else:
            
            # Log: Mensagem de diretorio existente
            print(str(datetime.now()) + ': Diretorio ' + directory + ' ja existe.')
    
    # Log: Mensagem de fim da criacao dos diretorios
    print(str(datetime.now()) + ': Criacao dos diretorios para armazenamento dos dados finalizada.')

In [42]:
def download_file(**kwargs):
    """
    Funcao que realiza o download do arquivo. Ao final do download, e verificado
    se o arquivo solicitado esta vazio. Caso esteja vazio, o arquivo e apagado. Caso contrario,
    o arquivo e descompactado.
    
    Args:
        url_query (str): URL parametrizada para download do arquivo.
        download_path (str): Caminho do diretorio onde sera salvo o arquivo.
        download_directory (str): Caminho do diretorio para decompactacao do arquivo
    """
    
    # Log: Mensagem de inicio do download do arquivo
    print(str(datetime.now()) + ': Inicio do download do arquivo ' + kwargs['download_path'])
    
    # Prepara a requisicao de download do arquivo
    res = req.get(kwargs['url_query'], stream = True)

    # Se for encontrada a URL para download
    if res.status_code == 200:
        
        # Faz o download do arquivo
        with open(kwargs['download_path'], 'wb') as f:
            
            # Log: Mensagem de URL encontrada e de inicio do download
            print(str(datetime.now()) + ': URL encontrada, fazendo o download do arquivo: ' + kwargs['download_path'])
            
            # Download do arquivo em pedacos
            for chunk in res.iter_content(chunk_size = 1024):
                f.write(chunk)
        
        # Verifica o tamanho do arquivo recebido
        file_size = os.stat(kwargs['download_path']).st_size
        
        # Se o tamanho do arquivo for muito 
        # pequeno remove o arquivo da pasta 
        if file_size <= 100:
            
            # Log: Mensagem de remocao por arquivo por nao conter dados
            print(str(datetime.now()) + ': Arquivo ' + kwargs['download_path'] + ' removido por conter poucos dados.')
            
            # Remove o arquivo sem dados
            os.remove(kwargs['download_path'])
            
        # Caso contrario, descompacta o arquivo csv 
        # na pasta e remove o arquivo compactado
        else:
            
            # Log: Mensagem de inicio da descompactacao do arquivo zip
            print(str(datetime.now()) + ': Descompactando o arquivo ' + kwargs['download_path'])
            
            # Descompacta o arquivo zip mantem apenas o csv
            zip_ref = zipfile.ZipFile(kwargs['download_path'], 'r')
            zip_ref.extractall(path=kwargs['download_directory'])
            zip_ref.close()
            os.remove(kwargs['download_path'])
            
            # Move os arquivos de dentro da pasta descompactada  
            # para a pasta de dados e remove a pasta vazia
            source_dir = kwargs['download_directory'] + kwargs['file_date_ref']
            if os.path.exists(source_dir):
                file_names = os.listdir(source_dir)
                for file_name in file_names:
                    shutil.move(os.path.join(source_dir, file_name), kwargs['download_directory'])
                os.rmdir(source_dir)
            
            # log: Mensagem de finalizacao do download do arquivo
            print(str(datetime.now()) + ': Fim do download do arquivo ' + kwargs['download_path'])

In [43]:
def collect_data(**kwargs):
    """
    Coleta os dados para cada ano de referencia dentro das datas de inicio e fim informadas.
    
    Args:
        dt_ini_ref (date): Data inicial para coleta dos dados
        dt_fim_ref (date): Data final para coleta dos dados
    """
    
    # Cria os diretorios para armazenamento dos dados
    data_directories()
    
    # Define a data de referencia inicial
    kwargs['dt_ref'] = kwargs['dt_ini_ref']
    
    # Define o diretorio para download dos arquivos
    kwargs['download_directory'] = '..\\data\\01-collected\\'
    
    # Define a URL base de busca dos arquivos para download
    kwargs['download_url'] = 'https://portal.inmet.gov.br/uploads/dadoshistoricos/'
    
    # Define a extensao do arquivo para download
    kwargs['file_ext'] = '.zip'
    
    # Log: Mensagem de inicio do processo 
    print(str(datetime.now()) + ': Coleta dos arquivos iniciada.')
       
    # Para cada mes de referencia dentro das datas de inicio e fim
    while kwargs['dt_ref'] <= kwargs['dt_fim_ref']:
        
        # Prepara as variaveis do mes de referencia
        kwargs['file_date_ref'] = kwargs['dt_ref'].strftime('%Y')
        kwargs['download_file'] = kwargs['file_date_ref'] + kwargs['file_ext']
        
        # Define a URL completa para download do arquivo
        kwargs['url_query'] = kwargs['download_url'] + kwargs['download_file']
        
        # Define o caminho completo para armazenamento do arquivo de download
        kwargs['download_path'] = kwargs['download_directory'] + kwargs['download_file']
        
        # Inicia o download do arquivo
        download_file(**kwargs)
        
        # Atualiza o mes de refencia
        kwargs['dt_ref'] += relativedelta(years=1)

    # Log: Mensagem de finalizacao do processo 
    print(str(datetime.now()) + ': Coleta dos arquivos finalizada.')

In [44]:
def main():
    """
    Coleta os dados meteorologicos das estacoes automaticas. 
    Origem: Instituto Nacional de Meteorologia
    """
    
    # Prepara o dicionario de variaveis (kwargs = keyworded arguments)
    kwargs = {}
    
    # Datas de Inicio e Fim (YYYY-MM-DD) 
    # Obs.: Dados disponiveis a partir de 2013-01-01
    kwargs['dt_ini_ref'] = datetime.strptime('2020-01-01', "%Y-%m-%d")
    kwargs['dt_fim_ref'] = datetime.strptime('2020-01-01', "%Y-%m-%d")
    
    # Coleta os arquivos
    collect_data(**kwargs)

if __name__ == '__main__':
    main()

2021-05-23 23:28:44.018124: Criacao dos diretorios para armazenamento dos dados iniciada.
2021-05-23 23:28:44.018124: Diretorio ..\data\01-collected ja existe.
2021-05-23 23:28:44.018124: Diretorio ..\data\02-cleaned ja existe.
2021-05-23 23:28:44.018124: Diretorio ..\data\03-organized ja existe.
2021-05-23 23:28:44.018124: Diretorio ..\data\04-standardized ja existe.
2021-05-23 23:28:44.018124: Criacao dos diretorios para armazenamento dos dados finalizada.
2021-05-23 23:28:44.018124: Coleta dos arquivos iniciada.
2021-05-23 23:28:44.018124: Inicio do download do arquivo ..\data\01-collected\2020.zip
2021-05-23 23:28:44.100975: URL encontrada, fazendo o download do arquivo: ..\data\01-collected\2020.zip
2021-05-23 23:28:47.564707: Descompactando o arquivo ..\data\01-collected\2020.zip
2021-05-23 23:28:50.074375: Fim do download do arquivo ..\data\01-collected\2020.zip
2021-05-23 23:28:50.075338: Coleta dos arquivos finalizada.
